# Step 1: Import Libraries

In [15]:
import cv2
import numpy as np
import os
import joblib
from skimage.feature import hog
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import albumentations as A
import matplotlib.pyplot as plt

# Step 2: Load Dataset & Extract Features

In [16]:
IMG_SIZE = (64, 64)  # Standardized Image Size

def load_data(dataset_path, classes):
    """Loads dataset, applies preprocessing, and extracts HOG features."""
    
    data, labels = [], []
    total_images = sum(len(os.listdir(os.path.join(dataset_path, str(i)))) for i in range(classes))

    with tqdm(total=total_images, desc="Extracting HOG Features", unit="img") as pbar:
        for i in range(classes):
            path = os.path.join(dataset_path, str(i))
            images = os.listdir(path)
            
            for img_name in images:
                try:
                    img_path = os.path.join(path, img_name)
                    image = cv2.imread(img_path, cv2.IMREAD_COLOR)

                    if image is None:
                        continue
                    
                    # Convert image to grayscale
                    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                    # Resize image
                    gray = cv2.resize(gray, IMG_SIZE)

                    # Compute HOG Features
                    features = hog(gray, pixels_per_cell=(8, 8), cells_per_block=(3, 3), feature_vector=True)
                    
                    data.append(features)
                    labels.append(i)
                
                except Exception as e:
                    print(f"Error loading {img_name}: {e}")
                
                pbar.update(1)
    
    return np.array(data), np.array(labels)

dataset_path = "gtsrb-german-traffic-sign/Train"
classes = 43  
X, y = load_data(dataset_path, classes)

print(f"Dataset Loaded: {len(X)} images with extracted HOG features.")

Extracting HOG Features: 100%|██████████| 39209/39209 [02:18<00:00, 283.15img/s]


Dataset Loaded: 39209 images with extracted HOG features.


# Step 3: Encode Labels & Standardize Features

In [17]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Labels Encoded & Features Standardized.")

Labels Encoded & Features Standardized.


# Step 4: Train-Test Split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

print(f"Data Split: {len(X_train)} training samples, {len(X_test)} testing samples.")


✅ Data Split: 31367 training samples, 7842 testing samples.


# Step 5: Train Optimized SVM Model

In [19]:
print("Training Optimized SVM Model...")

svm_model = SGDClassifier(loss='hinge', max_iter=5000, tol=1e-4, alpha=0.00001, 
                          learning_rate='adaptive', eta0=0.0005, 
                          penalty='elasticnet', l1_ratio=0.25, n_jobs=-1)

svm_model.fit(X_train, y_train)

# Train model
svm_model.fit(X_train, y_train)

print("Training Complete!")

# Step 6: Save Model & Preprocessors

In [ ]:
joblib.dump(svm_model, "hog_svm_traffic_signs_fast.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")
joblib.dump(scaler, "scaler.pkl")

print("Model & Preprocessing Files Saved!")

# Step 7: Evaluate Model Performance

In [ ]:
y_test_pred = svm_model.predict(X_test)

accuracy = accuracy_score(y_test, y_test_pred)
print(f"Final Model Accuracy: {accuracy:.4f}")


# Step 8: Define Function to Recognize a Traffic Sign

In [ ]:
def recognize_sign_hog(image, svm_model, scaler, label_encoder):
    """Recognizes traffic signs using HOG + SVM classification."""
    
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Resize image
    gray = cv2.resize(gray, IMG_SIZE)

    # Apply augmentation for testing
    transform = A.Compose([
        A.RandomBrightnessContrast(p=0.3),
        A.Rotate(limit=15, p=0.5),
    ])
    augmented = transform(image=gray)["image"]

    # Extract HOG Features
    features = hog(augmented, pixels_per_cell=(8, 8), cells_per_block=(3, 3), feature_vector=True)
    features = scaler.transform([features])

    # Predict with SVM
    predicted_label = svm_model.predict(features)[0]
    recognized_sign = label_encoder.inverse_transform([predicted_label])[0]

    return recognized_sign

# Step 9: Test Model on a New Image

In [ ]:
svm_model = joblib.load("hog_svm_traffic_signs_fast.pkl")
label_encoder = joblib.load("label_encoder.pkl")
scaler = joblib.load("scaler.pkl")

test_image_path = "gtsrb-german-traffic-sign/Test/00000.png"
test_image = cv2.imread(test_image_path)

if test_image is not None:
    recognized_sign = recognize_sign_hog(test_image, svm_model, scaler, label_encoder)
    print(f"Recognized Sign: {recognized_sign}")
else:
    print("Test Image Not Found.")